In [ ]:
import numpy as np       # linear algebra
import pandas as pd      # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf  # lib AI and numerical computing with tensores

# keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, ZeroPadding2D
from keras.models import Sequential
from keras.optimizers import RMSprop

from matplotlib import pyplot as plt

from keras.utils.np_utils import to_categorical # data processing - convert to one-hot-encoding

import matplotlib.pyplot as plt # visualization
%matplotlib inline

import os # comunication with operational system
print(os.listdir("../input"))

## Conjunto de Dados

Meu conjunto de dados se escontra na seguinte estrutura:
   *  _train.csv_: possui os dados de treinamento, com a primeira coluna possuindo as classes, ou rótulos, e as demais colunas possuem os valores dos pixeis da imagem, as características.
   * _test.csv_ : possui somente os valores dos pixeis, os labels serão previstos após a classificação do modelo de aprendizado de máquina definido.
   
   


In [ ]:
# lendo o dataset 
data = pd.read_csv('../input/train.csv')

# labels
x = data.drop(labels=["label"], axis=1) 

# features
y = data['label']

# liberando mais espaco
del data
print(x.shape, y.shape)

As imagens estão em uma dimensão, um vetor de tamanho N. Para tranfomar em uma imagem o vetor será redimensionado para  uma matrix 28 x 28.

In [ ]:
# redimensiomado o array 784 em matrix de 28 x 28 em uma canal, imagem em tons de cinza
x = x.values.reshape(-1,28,28,1)
x.shape

## Visualizando as imagens

In [ ]:
fig,ax = plt.subplots(1,4, figsize=(12,5))

for i in range(4):
    ax[i].imshow(x[i][:,:,0], cmap="gray")

## Normalização dos Dados 


In [ ]:
# as imagens são formadas de valores de 0 a 255, com essa rapida normalização, elas ficam na escala entre 0 e 1
x = x / 255.0

# é melhor para o modelo de aprendizado de máquina convergir valores de 0 a 1, do que 1 a 10, o aprendizado se torna mais rápido
y = to_categorical(y, num_classes=10)
print(x.shape, y.shape)

##  Separação dos Dados para Treinamento e Teste

Para treinar e testar o modelo de aprendizado de máquina, serão definido 4 variáveis:
   * **y_train**:  Contém os valores das classes, responsável para ser a saída desejada do conjunto _x_train_. 
   * **x_train**: Contém somente os valores do pixeis, é o conjunto de características que ensina ao modelo o que aquela classe é composta, a rede não aprende os valores do pixeis, mas um padrão em comum nesses dados.  _x_train_ e _y_train_ são usados somente na etapa de treinamento.
   * **x_test**: semelhate a _x_train_, mas se difere por serem imagens distintas das usadas pra treinamento e são usadas somente na etapa de teste.  o Intuito em usar dados diferentes aos treinados é avaliar a capacidade de generalização da rede, ou seja, ver o quão bom meu modelo é preciso em classificar dados com representações distintas da mesma classe.
   * **y_test**: semelhate a _y_train_, mas se difere por ser usado somente na etapa de teste e é utilizado para comparar com a saída da classificação do modelo, e validar quais predições ele classificou corretamente.
   
 Na biblioteca _sklearn_ tem um função que realiza essa serapação informando  a porcentagem opcional dos dados de teste e treino.
 



In [ ]:
from sklearn.model_selection import train_test_split
# divide os dados de treino e validacao para setar no treinamento
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
del x
del y

## Definindo o modelo de aprendizado de máquina

O bloco de construção básico de uma rede neural é a camada. As camadas extraem representações dos dados alimentados nelas. E, esperançosamente, essas representações são mais significativas para o problema em questão. O modelo foi contruído com as sequintes camadas: 

 * A primeira camada **Flatten**, transforma o formato das imagens de uma matriz 2d (de 28 por 28 pixels), para uma matriz 1d de 28 * 28 = 784 pixels. Essa camada não tem parâmetros para aprender; só reformata os dados.

 * A camada **Dense** é a camda oculta totalmente conectadas. A primeira camada Densa possui 128 nós (ou neurônios). A segunda (e última) camada é uma camada softmax de 10 nós - isso retorna uma matriz de 10 pontuações de probabilidade que somam 1. Cada nó contém uma pontuação que indica a probabilidade de a imagem atual pertencer a uma das 10 classes.
 
 * **Dropout** consiste em configurar aleatoriamente uma taxa de fração de unidades de entrada para 0 a cada atualização durante o tempo de treinamento, o que ajuda a evitar o overfitting. 



In [ ]:
def simpleCNN(entrada, weights_path=None):
    
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=entrada))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation="softmax"))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [ ]:

# definindo o model
model = simpleCNN(x_train[1].shape)

## Compilando o  modelo

Antes de o modelo estar pronto para o treinamento, ele precisa de mais algumas configurações. Estes são adicionados durante a etapa de compilação do modelo:
 * **Loss Function**: Isso mede a precisão do modelo durante o treinamento. Queremos minimizar essa função para "guiar" o modelo na direção certa.
 * **Optimizer** - é assim que o modelo é atualizado com base nos dados que ele vê e na sua função de perda.
 * **Metrics**: Usado para monitorar as etapas de treinamento e teste. O exemplo a seguir usa _accuracy_, a fração das imagens que são classificadas corretamente.


In [ ]:

model.compile(RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Treinando o modelo
Para começar o treinamento, chame o método _model.fit_ - o modelo é "adequado" aos dados de treinamento passando **x_train** e **y_train**. 

In [ ]:
print('-'*30)
print('treinando o modelo')
log = model.fit(x_train, y_train, batch_size=10, epochs=3)


In [ ]:
log.history.keys()

## Visualizado Acurácia e Perda no Treinamento

In [ ]:
# summarize history for loss
plt.plot(log.history['acc'], '--go')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')

In [ ]:
# summarize history for loss
plt.plot(log.history['loss'], '--ro')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

## Avaliando o Modelo com os Dados de Teste

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
print('Accuracy: %.2f ' %(test_acc*100))
print('Loss: %.2f '  %(test_loss*100))

Acontece que a precisão no conjunto de dados de teste é um pouco menor que a precisão no conjunto de dados de treinamento. Essa lacuna entre a precisão do treinamento e a precisão do teste é um exemplo de overfitting. O overfitting é quando um modelo de aprendizado de máquina apresenta um desempenho pior em novos dados do que em seus dados de treinamento.

## Fazendo a classificação

In [ ]:
# imagens utilizadas para teste

imagens_selecionadas = x_test[0:20,:,:,]

fig,ax = plt.subplots(1,imagens_selecionadas.shape[0], figsize=(12,5))

for i in range(imagens_selecionadas.shape[0]):
    ax[i].imshow(imagens_selecionadas[i,:,:,0], cmap="gray")

In [ ]:
# fazendo a predição sobre os dados de teste
predicao = model.predict(imagens_selecionadas)

## Visualizando a matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt =  'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
np.argmax(imagens_selecionadas)

y_pred = []
corretas = []

for i in range(len(imagens_selecionadas)):
    y_pred.append(np.argmax(predicao[i]))
    corretas.append(np.argmax(y_test[i]))
    
cnf_matrix = confusion_matrix(corretas, y_pred)
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix, classes=['1','2','3','4','5','6','7','8','9'])

## Fazendo a prvisão e submetendo o aquivo de submissão

In [ ]:
# lendo o dataset 
data = pd.read_csv('../input/test.csv')
data.head()

In [ ]:
# tranformando o vetor em imagem
test = data.values.reshape(-1,28,28,1)
test.shape

In [ ]:
# normalizando os dados de teste
test = test /  255.0

# fazendo a prediação
previsoes_test = model.predict(test)

In [ ]:
previsoes_test_label = []

for i in range(len(previsoes_test)):
    previsoes_test_label.append(np.argmax(previsoes_test[i]))

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(previsoes_test_label)+1)),
                         "Label": previsoes_test_label})
submissions.to_csv("DR.csv", index=False, header=True)

Em busca de aumentar, seus conhecimentos de aprendizado de máquina ? Vote no **kernel** para incentivar o desenvolvimento de mais problemas.

## Referências
 * [Get Started with TensorFlow](https://www.tensorflow.org/tutorials/)
 * [Matrix de confusão](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)
 * [NN Beginners](https://medium.com/@judyshih318/use-keras-to-build-the-first-simple-neural-network-as-a-beginner-8f0f2f6427e0)